In [0]:
%sql
SELECT * FROM dbxmetagen.metadata_results.comment_metadata_generation_log ORDER BY _created_at DESC;

In [0]:
%sql
DROP VIEW dbxmetagen.test_data.calculated_values_v;

In [0]:
%sql
CREATE OR REPLACE VIEW dbxmetagen.test_data.calculated_values_v AS 
SELECT id, 
nct_id, 
number_of_facilities, 
were_results_reported
FROM dbxmetagen.test_data.calculated_values;


In [0]:
%sql
ALTER TABLE dbxmetagen.test_data.calculated_values ALTER COLUMN `id` COMMENT "TEST ID";

In [0]:
%sql
ALTER VIEW dbxmetagen.test_data.calculated_values_v ALTER COLUMN `id` COMMENT "TEST ID";

In [0]:
df = spark.sql("DESCRIBE EXTENDED dbxmetagen.test_data.calculated_values_v;").toPandas()